# Investigating Amount of Necessary Real-World Data for Road-Following Tasks

#### Imports

In [45]:
import torch
import os
import importlib
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import image_dataset

importlib.reload(image_dataset)

from image_dataset import ImageSteeringAngleDataset, load_real_data, load_sim_data, shuffle_real_sim_data
from model import NeighborhoodRealCNN, HallwayCNN
from utils_graphs import plot_two_datasets, plot_model_sim_output, plot_loss_curve
import torchvision.transforms as transforms


#### Hyperparameters

In [46]:
sim_ratio = 0.5
data_sim_dir = "C:/Users/Cleah/Documents/AirSim/"
data_real_dir = "C:\\Users\\Cleah\\Documents\\Projects\\University Research\\Robot Learning Lab\\Simulator\\airsim-car-experiments\\PythonClient\\reinforcement_learning\\balanced_data_split\\"

#### Data Loading

In [47]:
# Load data. 
# Specify ratio of real:sim. 1 - sim_ratio = real_ratio
sim_ratio = 0.5

data_real_list = [f"{data_real_dir}"]
data_sim_list =[f"{data_sim_dir}2023-07-20-12-44-49",
                f"{data_sim_dir}2023-07-20-15-11-35",
                f"{data_sim_dir}2023-08-31-12-43-09",
                f"{data_sim_dir}2023-08-31-17-38-56",
                f"{data_sim_dir}2023-08-31-17-46-35",
                f"{data_sim_dir}2023-08-31-17-58-47",
                f"{data_sim_dir}2023-08-31-18-25-48",
                f"{data_sim_dir}2023-08-31-18-38-10",
                f"{data_sim_dir}2023-09-05-10-46-44",
                f"{data_sim_dir}2023-09-05-17-52-22",
                f"{data_sim_dir}2023-09-05-18-15-04",
                f"{data_sim_dir}2023-09-07-11-39-09",
                f"{data_sim_dir}2023-09-08-08-26-58",
                f"{data_sim_dir}2023-09-08-08-33-30",
                f"{data_sim_dir}2023-09-08-08-43-51",
                f"{data_sim_dir}2023-09-08-09-37-12",
                f"{data_sim_dir}2023-09-08-11-44-53",
                f"{data_sim_dir}2023-09-08-11-49-02",
                f"{data_sim_dir}2023-09-08-11-53-42",
                f"{data_sim_dir}2023-09-08-11-55-47",
                f"{data_sim_dir}2023-09-12-10-26-49"
            ]

real_data = load_real_data(data_real_list)
sim_data = load_sim_data(data_sim_list)
shuffled_real_sim_data = shuffle_real_sim_data(real_data, sim_data, sim_ratio)

dataset = ImageSteeringAngleDataset(shuffle_real_sim_data[0], shuffle_real_sim_data[1])

TypeError: Input tensor should be a torch tensor. Got <class 'builtin_function_or_method'>.

#### Train

#### Evaluate